In [5]:
import numpy as np

def findEuclideanDistance(source_representation, target_representation):
    """
    주어진 두 벡터의 유클리드 거리를 계산하는 함수
    
    Args:
    - source_representation : 1차원 numpy 배열. 벡터의 차원은 같아야 합니다.
    - test_representation : 1차원 numpy 배열. 벡터의 차원은 같아야 합니다.
    
    Returns:
    - distance : 주어진 두 벡터의 유클리드 거리
    """
    
    # 벡터 차원이 다른 경우 에러 메시지 출력
    if source_representation.shape != target_representation.shape:
        raise ValueError("두 벡터의 차원이 다릅니다.")
    
    # 각 원소의 차이를 계산한 후 제곱합니다.
    diff = source_representation - target_representation
    squared_diff = diff**2
    
    # 각 차이들의 합을 계산한 후 제곱근을 취해 거리를 계산합니다.
    distance = np.sqrt(np.sum(squared_diff))
    
    return distance

In [6]:
import math
from PIL import Image

def alignment_procedure(img, right_eye, left_eye):

    # this function aligns given face in img based on left and right eye coordinates

    right_eye_x, right_eye_y = right_eye
    left_eye_x, left_eye_y = left_eye

    # -----------------------
    # find rotation direction

    if right_eye_y > left_eye_y:
        point_3rd = (left_eye_x, right_eye_y)
        direction = -1  # rotate same direction to clock
    else:
        point_3rd = (right_eye_x, left_eye_y)
        direction = 1  # rotate inverse direction of clock

    # -----------------------
    # find length of triangle edges

    a = findEuclideanDistance(np.array(right_eye), np.array(point_3rd))
    b = findEuclideanDistance(np.array(left_eye), np.array(point_3rd))
    c = findEuclideanDistance(np.array(left_eye), np.array(right_eye))

    # -----------------------

    # apply cosine rule

    if b != 0 and c != 0:  # this multiplication causes division by zero in cos_a calculation

        cos_a = (b * b + c * c - a * a) / (2 * b * c)
        angle = np.arccos(cos_a)  # angle in radian
        angle = (angle * 180) / math.pi  # radian to degree

        # -----------------------
        # rotate base image

        if direction == -1:
            angle = 90 - angle

        img = Image.fromarray(img)
        img = np.array(img.rotate(direction * angle))

    # -----------------------

    return img  # return img anyway

In [10]:
import cv2
import mediapipe as mp
import os
from pathlib import Path

# NVIDIA 그래픽 카드가 있는 경우, 이 옵션 추가하기 전에는 18분 48.2초 걸림 -> 18분 17.7초
if cv2.cuda.getCudaEnabledDeviceCount() > 0:
    cv2.cuda.setDevice(0)

def get_last_path(path):
    splitted = str(path).replace('\\','/').split('/')
    if not splitted[-1]:
        return splitted[-2]
    return splitted[-1]

def get_after_specific_path(input_path, string):
    splitted = str(input_path).replace('\\','/').split('/')
    index = splitted.index(string)
    result = '/'.join(splitted[index + 1:])
    return result

def process_and_save_faces(input_path, output_path, input_base, output_base):
    originImg_path_list = []
    extractedImg_path_list = []
    xywh_list = []
    score_list = []
    new_order_list = []

    # 이미지 읽기
    image = cv2.imread(str(input_path))  # cv2가 Path가 아닌 str으로 경로를 받음

    # MediaPipe를 사용하여 얼굴 좌표 추출
    results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # 추출된 얼굴 좌표를 사용하여 얼굴 이미지 잘라내기 및 리사이즈
    if results.detections:
        order_list = []
        for idx, detection in enumerate(results.detections):
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = image.shape
            x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
            cropped_face = image[y:y+h, x:x+w]
            
            # 이미지를 PNG 형식으로 저장 - 01 cropped
            save_filename = str(os.path.join(f"{output_path}_{idx:0>3}_01cropped.png"))
            cv2.imwrite(save_filename, cropped_face, [cv2.IMWRITE_PNG_COMPRESSION, 9])

            landmarks = detection.location_data.relative_keypoints

            right_eye = (int(landmarks[0].x * iw), int(landmarks[0].y * ih))
            left_eye = (int(landmarks[1].x * iw), int(landmarks[1].y * ih))

            aligned_face = alignment_procedure(cropped_face, right_eye, left_eye)

            # 이미지를 PNG 형식으로 저장 - 02 aligned
            save_filename = str(os.path.join(f"{output_path}_{idx:0>3}_02aligned.png"))
            cv2.imwrite(save_filename, aligned_face, [cv2.IMWRITE_PNG_COMPRESSION, 9])
            
            # 리사이즈: 비율 유지를 위해 패딩(padding) 추가
            face_height, face_width, _ = aligned_face.shape
            if face_width > face_height:
                padding = int((face_width - face_height) / 2)
                padded_face = cv2.copyMakeBorder(aligned_face, padding, padding, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
            else:
                padding = int((face_height - face_width) / 2)
                padded_face = cv2.copyMakeBorder(aligned_face, 0, 0, padding, padding, cv2.BORDER_CONSTANT, value=(0, 0, 0))
            
            # 이미지를 PNG 형식으로 저장 - 04 resized
            save_filename = str(os.path.join(f"{output_path}_{idx:0>3}_03padded.png"))
            cv2.imwrite(save_filename, padded_face, [cv2.IMWRITE_PNG_COMPRESSION, 9])

            # 얼굴 이미지를 224x224로 리사이즈
            resized_face = cv2.resize(padded_face, (224, 224))
            
            # 이미지를 PNG 형식으로 저장 - 04 resized
            save_filename = str(os.path.join(f"{output_path}_{idx:0>3}_04resized.png"))
            cv2.imwrite(save_filename, resized_face, [cv2.IMWRITE_PNG_COMPRESSION, 9])
            # originImg_path_list.append(get_after_specific_path(input_path, get_last_path(input_base)))
            originImg_path_list.append("")
            # extractedImg_path_list.append(get_after_specific_path(save_filename, get_last_path(output_base)))
            extractedImg_path_list.append("")
            xywh_list.append((x, y, w, h))
            score_list.append(detection.score[0])
            order_list.append(idx)
        new_order_list = [str(i+1) + '/' + str(len(order_list)) for i in order_list]

    else:
        # 첫 항목인 경우는 
        # originImg_path_list.append(get_after_specific_path(input_path, get_last_path(input_base)))
        originImg_path_list.append("")
        # extractedImg_path_list.append('(No Face)')
        extractedImg_path_list.append("")
        xywh_list.append('(No Face)')
        score_list.append('(No Face)')
        new_order_list.append(0)
    return originImg_path_list, extractedImg_path_list, xywh_list, score_list, new_order_list


# MediaPipe 얼굴 인식을 위한 객체 생성
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.2)


## 이곳에서 데이터 인풋과 아웃풋위치를 변경해주시면 됩니다.###########################################################################################
input_base_path = '../../../datasets/_temp/base'
output_base_path = '../../../datasets/_temp/face'

input_base = Path(input_base_path)
output_base = Path(output_base_path)

originImg_path_list = []
extractedImg_path_list = []
xywh_list = []
score_list = []
order_list = []
num = 0
for path in input_base.rglob('*.JPG'):  # 모든 JPG 파일 검색 (대소문자 구분 없이 검색하려면 '*.jpg'를 '*.JPG'로 변경)
    relative_path = path.relative_to(input_base)
    # print(type(relative_path))
    output_path = output_base / relative_path.with_suffix('')
    
    # 상위 디렉토리부터 하위 디렉토리 생성
    os.makedirs(output_path.parent, exist_ok=True)

    # 얼굴 이미지 처리 및 저장
    originImg_path_array, extractedImg_path_array, xywh_array, score_array, order_array = process_and_save_faces(path, output_path, input_base, output_base)

    originImg_path_list.extend(originImg_path_array)
    extractedImg_path_list.extend(extractedImg_path_array)
    xywh_list.extend(xywh_array)
    score_list.extend(score_array)
    order_list.extend(order_array)
    

In [1]:
import cv2
cv2.__version__

'4.7.0'